# Debugging PiNN layers and networks [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Teoroo-CMC/PiNN/blob/master/docs/notebooks/Layer_debug.ipynb)

In [5]:
# !pip install git+https://github.com/Teoroo-CMC/PiNN
!wget -nv -nc https://raw.githubusercontent.com/Teoroo-CMC/PiNN_lab/master/resources/qm9_train.{yml,tfr}

In [1]:
%load_ext autoreload
%autoreload 2

## Loading data

For the purpose of testing we download a subset of the QM9 dataset used in [PiNN_lab](https://github.com/Teoroo-CMC/PiNN_lab).

In [2]:
from pinn.io import load_tfrecord, sparse_batch

dataset = load_tfrecord("qm9_train.yml").apply(sparse_batch(10))
for datum in dataset:
    print({k: v.shape for k, v in datum.items()})
    break

2023-10-09 15:48:59.608727: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-09 15:48:59.637897: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-09 15:48:59.637947: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-09 15:48:59.637968: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-09 15:48:59.644280: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-09 15:48:59.645407: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

{'A': TensorShape([10]), 'B': TensorShape([10]), 'C': TensorShape([10]), 'Cv': TensorShape([10]), 'G': TensorShape([10]), 'H': TensorShape([10]), 'U': TensorShape([10]), 'U0': TensorShape([10]), 'alpha': TensorShape([10]), 'coord': TensorShape([176, 3]), 'elems': TensorShape([176]), 'gap': TensorShape([10]), 'homo': TensorShape([10]), 'lumo': TensorShape([10]), 'mu': TensorShape([10]), 'r2': TensorShape([10]), 'zpve': TensorShape([10]), 'ind_1': TensorShape([176, 1])}


## Using PiNN Layers

PiNN networks and layers are Keras [Layers](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer) 
and [Models](https://www.tensorflow.org/api_docs/python/tf/keras/Model). 

To use them, you create an instance of layer, after that, the layer object can be used as a function. Each layer
is initialized with different parameters and requires different input tensors, see their individual documentation
for the details.

In [37]:
from pinn.layers import CellListNL
from pinn.networks.pinet2 import PILayer, PiNet2
def label_data(data):
    # defines the label to train on
    x = data
    y = data['lumo']
    return x, y

train = dataset.map(label_data)
pinet = PiNet2(out_pool='min')

#pinet(next(iter(train))[0])
pinet.compile(optimizer='Adam', loss='MAE')
pinet.fit(train, epochs=3)

[176 3 16]
[176 3 16]
[176 3 16]
[176 3 16]


<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([-0.09588242,  0.1565401 , -0.8049977 , -0.34934044, -0.9046093 ,
       -1.272258  , -0.3640451 , -0.64601314, -1.4608136 , -0.2622609 ],
      dtype=float32)>

In [33]:
next(iter(train))[0]['ind_1'].shape

TensorShape([176, 1])

In [15]:
import tensorflow as tf

class PiNet2(tf.keras.Model):
    def __init__(self):
        super(PiNet2, self).__init__()

PiNet2()

## Further benchmarks

For more advanced usage you are recommended to use the `Model` API to define the trainig loss, derived predicates.
For traininig potential energy surfaces, you are recommended to use `pinn.models.potential_model` in combination with the command line interface (CLI).

Alternatively, see the [Trainig Tips](More_on_training.ipynb) notebook to see how to run the tranining interactively in a notebook.